In [1]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
from pyspark.sql import functions as f

StatementMeta(tcccibele, 8, 1, Finished, Available)

In [2]:
# Definir schema do arquivo
schema = StructType([ 
    StructField("None", StringType(), True), 
    StructField("RacaSuspeito", StringType(), True), 
    StructField("Janeiro", StringType(), True), 
    StructField("Fevereiro", StringType(), True), 
    StructField("Março", StringType(), True), 
    StructField("Abril", StringType(), True), 
    StructField("Maio", StringType(), True), 
    StructField("Junho", StringType(), True), 
    StructField("Julho", StringType(), True), 
    StructField("Agosto", StringType(), True), 
    StructField("Setembro", StringType(), True), 
    StructField("Outubro", StringType(), True), 
    StructField("Novembro", StringType(), True), 
    StructField("Dezembro", StringType(), True), 
    StructField("Total", StringType(), True)
])

StatementMeta(tcccibele, 8, 2, Finished, Available)

In [3]:
# Ler arquivo de origem da landing zone
df = spark.read.format("csv") \
            .schema(schema) \
            .options(quote='"') \
            .load('/landing_zone/2019/Balanco-2019-Ligue180-Raça-cor do Suspeito.csv')

StatementMeta(tcccibele, 8, 3, Finished, Available)

In [4]:
# Remover colunas e linhas sem valor para o data lake
df = df.where((df[1].isNotNull()) \
                & (df[1] != 'Total') \
                & (df[-1] != 'Total 2019') \
                & (df[-1].isNotNull()))
df = df.drop('None')
df = df.drop('Total')

StatementMeta(tcccibele, 8, 4, Finished, Available)

In [5]:
# Remover "," de todos os campos e substitui null por 0
names = df.schema.names
for name in names:
    if name == 'RacaSuspeito':
        continue
    df = df.withColumn(name,f.regexp_replace(name,',',''))
    df = df.withColumn(name, df[name].cast(IntegerType()))

df = df.fillna(0)

StatementMeta(tcccibele, 8, 5, Finished, Available)

In [6]:
# Escrever dados limpos em formato parque no azure storage
df.write.format('parquet').mode('overwrite').save('/history_zone/2019/RacaSuspeito/')

StatementMeta(tcccibele, 8, 6, Finished, Available)

In [7]:
df = spark.read.parquet('/history_zone/2019/RacaSuspeito/')
df.toPandas()

StatementMeta(tcccibele, 8, 7, Finished, Available)

,RacaSuspeito,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro
0,Amarela,93,72,73,62,51,42,61,58,50,52,29,20
1,Branca,3344,2616,2896,2464,2472,2240,2326,2631,2563,2609,2347,1303
2,Indígena,44,45,50,50,43,38,34,50,41,38,32,20
3,Parda,3166,2708,2954,2640,2657,2289,2301,2680,2532,2657,2380,1289
4,Preta,1328,1026,1212,1110,1114,921,965,1125,1056,1005,1060,483
5,Não Informado,1574,1202,1255,917,972,770,755,1084,1070,919,840,437
